In [2]:
# Import the dependencies.
import numpy as np
import pandas as pd
import datetime as dt

# from matplotlib import style
# style.use('fivethirtyeight')
# import matplotlib.pyplot as plt

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

from flask import Flask, jsonify

In [3]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(autoload_with=engine)
# View all of the classes that automap found
Base.classes.keys()
# Save references to each table
Measurements = Base.classes.measurement
Stations = Base.classes.station
# Create our session (link) from Python to the DB
session = Session(bind=engine)


In [8]:
#latitude,longitude,elevation
wx_stations = session.query(Stations.station, Stations.name, Stations.latitude, Stations.longitude, Stations.elevation).all()
all_stations = []
for row in wx_stations:
    wx_stations_dict = {}
    wx_stations_dict["ID:"] = row.station
    wx_stations_dict["Name:"] = row.name
    wx_stations_dict["Lat:"] = row.latitude
    wx_stations_dict["Long:"] = row.longitude
    wx_stations_dict["Elevation:"] = row.elevation
    all_stations.append(wx_stations_dict)
print(all_stations)

[{'ID:': 'USC00519397', 'Name:': 'WAIKIKI 717.2, HI US', 'Lat:': 21.2716, 'Long:': -157.8168, 'Elevation:': 3.0}, {'ID:': 'USC00513117', 'Name:': 'KANEOHE 838.1, HI US', 'Lat:': 21.4234, 'Long:': -157.8015, 'Elevation:': 14.6}, {'ID:': 'USC00514830', 'Name:': 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 'Lat:': 21.5213, 'Long:': -157.8374, 'Elevation:': 7.0}, {'ID:': 'USC00517948', 'Name:': 'PEARL CITY, HI US', 'Lat:': 21.3934, 'Long:': -157.9751, 'Elevation:': 11.9}, {'ID:': 'USC00518838', 'Name:': 'UPPER WAHIAWA 874.3, HI US', 'Lat:': 21.4992, 'Long:': -158.0111, 'Elevation:': 306.6}, {'ID:': 'USC00519523', 'Name:': 'WAIMANALO EXPERIMENTAL FARM, HI US', 'Lat:': 21.33556, 'Long:': -157.71139, 'Elevation:': 19.5}, {'ID:': 'USC00519281', 'Name:': 'WAIHEE 837.5, HI US', 'Lat:': 21.45167, 'Long:': -157.84888999999998, 'Elevation:': 32.9}, {'ID:': 'USC00511918', 'Name:': 'HONOLULU OBSERVATORY 702.2, HI US', 'Lat:': 21.3152, 'Long:': -157.9992, 'Elevation:': 0.9}, {'ID:': 'USC00516128', 'Name:

In [16]:
start_date = "2017-04-04"
min_max_avg = session.query(Stations.station, Stations.name,\
    func.min(Measurements.tobs), func.max(Measurements.tobs), func.round(func.avg(Measurements.tobs))).\
    filter(Measurements.station == Stations.station).filter(Measurements.date >= start_date).all()
for row in min_max_avg:
    print(row)
# min_max_avg
# all_stations_temps = []
# for row in wx_stations:
#     wx_stations_temp_dict = {}
#     wx_stations_temp_dict["ID"] = row.station
#     wx_stations_temp_dict["Name"] = row.name
#     wx_stations_temp_dict["Min"] = row.latitude
#     wx_stations_temp_dict["Max"] = row.longitude
#     wx_stations_temp_dict["AVG"] = row.elevation
#     all_stations.append(wx_stations_dict)
# print(all_stations_temps)


('USC00519397', 'WAIKIKI 717.2, HI US', 58.0, 87.0, 76.0)


In [21]:
start_date = '2011-04-04'
min_max_avg = session.query(Stations.station, Stations.name, func.min(Measurements.tobs), \
            func.max(Measurements.tobs), func.round(func.avg(Measurements.tobs))).group_by(Stations.station).\
            filter(Measurements.station == Stations.station).filter(Measurements.date >= start_date).all()

station_list_temp = [(*result,) for result in min_max_avg]
print(station_list_temp)

[('USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 53.0, 86.0, 72.0), ('USC00513117', 'KANEOHE 838.1, HI US', 59.0, 85.0, 73.0), ('USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 63.0, 85.0, 75.0), ('USC00516128', 'MANOA LYON ARBO 785.2, HI US', 58.0, 84.0, 71.0), ('USC00517948', 'PEARL CITY, HI US', 59.0, 87.0, 75.0), ('USC00518838', 'UPPER WAHIAWA 874.3, HI US', 58.0, 80.0, 74.0), ('USC00519281', 'WAIHEE 837.5, HI US', 54.0, 85.0, 72.0), ('USC00519397', 'WAIKIKI 717.2, HI US', 56.0, 87.0, 75.0), ('USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 56.0, 85.0, 75.0)]


In [23]:
station_dict_temp = {result[0]: {"name": result[1], "min_temp": result[2], "max_temp": result[3],\
                         "avg_temp": result[4]} for result in min_max_avg}
print(station_dict_temp)

{'USC00511918': {'name': 'HONOLULU OBSERVATORY 702.2, HI US', 'min_temp': 53.0, 'max_temp': 86.0, 'avg_temp': 72.0}, 'USC00513117': {'name': 'KANEOHE 838.1, HI US', 'min_temp': 59.0, 'max_temp': 85.0, 'avg_temp': 73.0}, 'USC00514830': {'name': 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 'min_temp': 63.0, 'max_temp': 85.0, 'avg_temp': 75.0}, 'USC00516128': {'name': 'MANOA LYON ARBO 785.2, HI US', 'min_temp': 58.0, 'max_temp': 84.0, 'avg_temp': 71.0}, 'USC00517948': {'name': 'PEARL CITY, HI US', 'min_temp': 59.0, 'max_temp': 87.0, 'avg_temp': 75.0}, 'USC00518838': {'name': 'UPPER WAHIAWA 874.3, HI US', 'min_temp': 58.0, 'max_temp': 80.0, 'avg_temp': 74.0}, 'USC00519281': {'name': 'WAIHEE 837.5, HI US', 'min_temp': 54.0, 'max_temp': 85.0, 'avg_temp': 72.0}, 'USC00519397': {'name': 'WAIKIKI 717.2, HI US', 'min_temp': 56.0, 'max_temp': 87.0, 'avg_temp': 75.0}, 'USC00519523': {'name': 'WAIMANALO EXPERIMENTAL FARM, HI US', 'min_temp': 56.0, 'max_temp': 85.0, 'avg_temp': 75.0}}


In [25]:
start = "2016-04-04"
end = "2016-12-22"
min_max_avg = session.query(Stations.station, Stations.name, func.min(Measurements.tobs), \
            func.max(Measurements.tobs), func.round(func.avg(Measurements.tobs))).group_by(Stations.station).\
            filter(Measurements.station == Stations.station).\
            filter(Measurements.date >= start).filter(Measurements.date <= end).all()
station_dict_temp = {result[0]: {"name": result[1], "min_temp": result[2], "max_temp": result[3],\
                         "avg_temp": result[4]} for result in min_max_avg}
print(station_dict_temp)

{'USC00513117': {'name': 'KANEOHE 838.1, HI US', 'min_temp': 63.0, 'max_temp': 79.0, 'avg_temp': 75.0}, 'USC00514830': {'name': 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 'min_temp': 69.0, 'max_temp': 83.0, 'avg_temp': 78.0}, 'USC00516128': {'name': 'MANOA LYON ARBO 785.2, HI US', 'min_temp': 60.0, 'max_temp': 81.0, 'avg_temp': 72.0}, 'USC00517948': {'name': 'PEARL CITY, HI US', 'min_temp': 64.0, 'max_temp': 82.0, 'avg_temp': 77.0}, 'USC00519281': {'name': 'WAIHEE 837.5, HI US', 'min_temp': 62.0, 'max_temp': 82.0, 'avg_temp': 75.0}, 'USC00519397': {'name': 'WAIKIKI 717.2, HI US', 'min_temp': 62.0, 'max_temp': 82.0, 'avg_temp': 77.0}, 'USC00519523': {'name': 'WAIMANALO EXPERIMENTAL FARM, HI US', 'min_temp': 63.0, 'max_temp': 84.0, 'avg_temp': 78.0}}


In [9]:
# Close Session
session.close()